In [ ]:
import cv2 as cv
import torch
import numpy as np
import os
import json
from torch import nn
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.datasets import VOCDetection
from torchvision.transforms import ToTensor

from datasets import YoloVOCDataset
from utils.my_dataset import YoloDataset
from model import Yolov1
from utils.draw import draw_box

In [ ]:
device='cpu'
IMG_SIZE=224
S=7
C=1

grid_size=IMG_SIZE/S

model=Yolov1(S,C).to(device)
model.load_state_dict(torch.load('checkpoint.pth',map_location=device)['model'])
model.eval()

# ds=YoloVOCDataset(IMG_SIZE,S,C)

In [5]:
path=os.path.join('data\VOCdevkit\VOC2012\JPEGImages','2007_000068.jpg')
img=cv.imread(path)

img=cv.resize(img,(IMG_SIZE,IMG_SIZE))
input=ToTensor()(img)

output=model(input.unsqueeze(0))[0]

for i in range(S):
    for j in range(S):
        if output[i,j,4]>0.5:
            draw_box(img,i,j,output[i,j,:5],IMG_SIZE,S)
        if output[i,j,9]>0.5:
            draw_box(img,i,j,output[i,j,5:10],IMG_SIZE,S)

img=cv.resize(img,(512,512))
cv.imshow('Camera', img)
cv.waitKey(0)
cv.destroyAllWindows()